# A* algorithm to find the shortest path for ground floor

-  Contributed by: Hardik Soni

In [13]:
#initialising the time to know the run time 
import time
start = time.time()

In [14]:
# This class represent a graph
class Graph:
    # Initialize the class
    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()
    # Create an undirected graph by adding symmetric edges
    def make_undirected(self):
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.graph_dict.setdefault(b, {})[a] = dist
    def connect(self, A, B, distance=1):
        self.graph_dict.setdefault(A, {})[B] = distance
        if not self.directed:
            self.graph_dict.setdefault(B, {})[A] = distance
    # Get neighbors or a neighbor
    def get(self, a, b=None):
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)
    # Return a list of nodes in the graph
    def nodes(self):
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)


In [15]:
# This class represent a node
class Node:
    # Initialize the class
    def __init__(self, name:str, parent:str):
        self.name = name
        self.parent = parent
        self.g = 0 # Distance to start node
        self.h = 0 # Distance to goal node
        self.f = 0 # Total cost
    # Compare nodes
    def __eq__(self, other):
        return self.name == other.name
    # Sort nodes
    def __lt__(self, other):
         return self.f < other.f
    # Print node
    def __repr__(self):
        return ('({0},{1})'.format(self.name, self.f))

In [16]:
# A* search
def astar_search(graph, heuristics, start, end):
    
    # Create lists for open nodes and closed nodes
    open = []
    closed = []
    # Create a start node and an goal node
    start_node = Node(start, None)
    goal_node = Node(end, None)
    # Add the start node
    open.append(start_node)
    
    # Loop until the open list is empty
    while len(open) > 0:
        # Sort the open list to get the node with the lowest cost first
        open.sort()
        # Get the node with the lowest cost
        current_node = open.pop(0)
        # Add the current node to the closed list
        closed.append(current_node)
        
        # Check if we have reached the goal, return the path
        if current_node == goal_node:
            path = []
            while current_node != start_node:
                path.append(current_node.name) #+ str(current_node.g)
                current_node = current_node.parent
            path.append(start_node.name)    #+ str(start_node.g)
            # Return reversed path
            return path[::-1]
        # Get neighbours
        neighbors = graph.get(current_node.name)
        # Loop neighbors
        for key, value in neighbors.items():
            # Create a neighbor node
            neighbor = Node(key, current_node)
            # Check if the neighbor is in the closed list
            if(neighbor in closed):
                continue
            # Calculate full path cost
            neighbor.g = current_node.g + graph.get(current_node.name, neighbor.name)
            neighbor.h = heuristics.get(neighbor.name)
            neighbor.f = neighbor.g + neighbor.h
            # Check if neighbor is in open list and if it has a lower f value
            if(add_to_open(open, neighbor) == True):
                # Everything is green, add neighbor to open list
                open.append(neighbor)
                
    print(neighbor.g)
    # Return None, no path is found
    return None


In [17]:
# Check if a neighbor should be added to open list
def add_to_open(open, neighbor):
    for node in open:
        if (neighbor == node and neighbor.f > node.f):
            return False
    return True

In [18]:
#intialising the nodes and random fire
nodes = ["Stairs1", "Stairs2", "L1", "L2", "L3", "L4", "L5", "Middle1", "Middle2", "Middle3", "Washroom1", "Washroom2", "EXIT2"]

import random
random_item_from_list = random.choice(nodes)
print("Fire is at: ")
print(random_item_from_list)

Fire is at: 
L1


In [19]:
#Datasets File
import pandas as pd
df = pd.read_csv (r"AI_datasets - Sheet1.csv")
print (df)
present = df.loc[df.Person != 0, 'Node_Places']

   Node_Places  Person
0      Stairs1       5
1      Stairs2       0
2           L1       0
3           L2       0
4           L3       0
5           L4       0
6           L5       0
7        EXIT1       0
8      Middle1       0
9      Middle2       0
10     Middle3       0
11       EXIT2       0
12   Washroom1       0
13   Washroom2       2


In [20]:
present = df.loc[df.Person != 0, 'Node_Places']
print(present)
#no. of nodes having Persons
n = len(present)

0       Stairs1
13    Washroom2
Name: Node_Places, dtype: object


In [21]:
def main():
    # Create a graph
    graph = Graph()
    # Create graph connections (Actual distance)
    graph.connect('Washroom1', 'L4', 2)
    graph.connect('Washroom1', 'Middle1', 3)
    graph.connect('Stairs1', 'Middle1', 3)
    graph.connect('L3', 'Middle1', 2)
    graph.connect('L3', 'Middle2', 2)
    graph.connect('L2', 'Middle2', 2)
    graph.connect('L2', 'Middle1', 2)
    graph.connect('Stairs2', 'Middle2', 3)
    graph.connect('Washroom2', 'Middle2', 3)
    graph.connect('Washroom2', 'L1', 2)
    graph.connect('Middle1', 'L4', 3)
    graph.connect('Middle1', 'Middle2', 1)
    graph.connect('Middle1', 'Middle3', 1)
    graph.connect('Middle2', 'Middle3', 1)
    graph.connect('Middle2', 'L1', 3)
    graph.connect('Middle3', 'L1', 2)
    graph.connect('L4', 'Middle3', 2)
    graph.connect('Middle3', 'EXIT2', 2)
    graph.connect('L1', 'EXIT2', 1)
    graph.connect('L5', 'EXIT2', 2)
    graph.connect('L5', 'EXIT1', 2)
    graph.connect('L4', 'EXIT1', 1)
    graph.connect('Middle3', 'EXIT1', 2)
    

    # Make graph undirected, create symmetric connections
    graph.make_undirected()
    # Create heuristics (straight-line distance, air-travel distance)
    heuristics = {}
    heuristics['Washroom1'] = 100
    heuristics['Stairs1'] = 98
    heuristics['L3'] = 95
    heuristics['L2'] = 93
    heuristics['Stairs2'] = 96
    heuristics['Washroom2'] = 89
    heuristics['Middle1'] = 85
    heuristics['Middle2'] = 78
    heuristics['L4'] = 90
    heuristics['Middle3'] = 60
    heuristics['L1'] = 45
    heuristics['EXIT2'] = 0
    heuristics['L5'] = 50
    heuristics['EXIT1'] = 0
    
    #removing the fire node
    if (heuristics[random_item_from_list]!=None):
        heuristics[random_item_from_list]=10000
    
    #Setting variable starting node

    # Run the search algorithm
    for i in present: 
        path = astar_search(graph, heuristics, i, 'EXIT2')
        if(random_item_from_list == 'EXIT2'):
            print('Path for ' +i)
            print('There is no way to escape, Rest in Peace! :)')
        
        else:    
                print('The best path obtained is given as :','\n',path)
        print()
    
    
# Tell python to run main method
if __name__ == "__main__": main()
    
end = time.time()    
print(f"Runtime of the program is {end - start} ms.")

The best path obtained is given as : 
 ['Stairs1', 'Middle1', 'Middle3', 'EXIT2']

The best path obtained is given as : 
 ['Washroom2', 'Middle2', 'Middle3', 'EXIT2']

Runtime of the program is 0.2687811851501465 ms.
